In [53]:
import numpy as np
import os
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib qt5
plt.rc('font', size=16)

from IPython import display

### CONSTANTS ###
m_u = 1.66054e-24 # unit mass (g)
a0 = 0.5291772109 # Bohr radius (Å)

# SIMULATING A BULK WATER SYSTEM USING CAR-PARRINELLO MOLECULAR DYNAMICS

<div style="text-align: center"><i>Alessandro Cuoghi - April 2023</i></div>

--- 

# *I* - INTRODUCTION
In studying the physics of complex systems, numerical simulations provide an essential tool to determine the values of quantities that are difficult to obtain through experimental techniques, as well as to verify theories or to examine how certain principles affect the dynamics of the system. From this perspective, studies performed via numerical simulations allow also to examine unphysical processes.

Classical molecular dynamics (MD) is a commonly used computational tool for simulating the properties of liquids, solids, and molecules. Each of the atoms or molecules in the simulation is treated as a point mass and Newton’s equations are integrated to compute their motion. In classical MD the forces are derived from empirical laws and the accuracy of the results strongly depends on the chosen model. This approach has enjoyed tremendous success in the treatment of systems ranging from simple liquids and solids to polymers and biological systems such as proteins and nucleic acids. Despite their success, force fields have a number of serious limitations, the major being that, first, these calculations convey no information about electronic properties and, second, force fields generally assume a pre-specified connectivity among the atoms and, therefore, suffer from an inability to describe chemical bond breaking and forming events. On the other hand, density functional theory (DFT) calculations have provided an accurate, albeit approximate, description of the chemical bond in a large variety of systems, altough they are very demanding computationally and it is only realistically possible to do small simulations for short times. This limits the application of DFT schemes to the study of very large and/or disordered systems and to the computation of interatomic forces for MD simulations.

# *II* - AB INITIO MOLECULAR DYNAMICS
The techniques of every MD scheme face with two differents problems:
- how to compute the trajectory by numerically integrating the classical equation of motion for every particle in the system.
- how to define a force field that correctly describes the physics of the system.
 
## *II.I* - INTEGRATING THE EQUATION OF MOTION. THE VERLET ALGORITHM
In order to evolve the position and the velocity of a particle and computen the trajectory, time is *discretized* in many steps of equal length $\Delta t$. Then, the position of every particle $I$ at a successive step $\vec R_I(t+\Delta t)$ depends on the position, velocity and force at the previous time step $t$

$$\vec R_I(t+\Delta t) = \vec R_I(t) + \vec v_I(t)\Delta t + \frac{\vec F_I(t)}{2m_I}\Delta t^2 + \mathcal{O}(\Delta t^3)$$

There exist many algorithms that allow to integrate Newton's equation of motion; one of the most used and the one that is implemented in the simulations for this report is the Verlet's algorithm [$^{[1]}$](#bib-ver) 

<a id='eq-ver'></a>
$$\vec R_I(t+\Delta t) = 2\vec R_I(t) - \vec R_I(t-\Delta t) + \frac{\vec F_I(t)}{2m_I}\Delta t^2 \tag{1}$$

for which is possible to demonstrate that has an error of $\mathcal{O}(\Delta t^4)$. As we want an efficient simulation that allows to compute quantities of interest with precise values, we want to introduce a numerical error that is as small as possible. Thus, it is crucial the choice of the timestep $\Delta t$: it should be very small so that to minimize the error of the integration procedure, but on the other hand, it should be big enough to allow the simulation to run for a given amount of time needed to simulate properly the physics we are interested in within the computational resources available. It turns out that for a Verlet scheme [(Eq. 1)](#eq-ver), a good choice for the timestep is to take the fastest characteristic time of the system and divide it by 30, $\Delta t \approx \tau/30$.

A general scheme for a MD simulation is composed of 4 steps:
1. *Initialize positions and velocities for every particle of the system*. The velocities are found straightforward by sampling the Boltzmann's distribution at a given temperature; the atomic positions must be carefully determined and chosen such that the system is in a relaxed configuration. Hence, most of the time it is a good idea to run a preliminary simulation with a classical force field to get a more or less thermalized initial state to start with. 

2. *Compute the forces and integrate the equation of motion*. As already discussed, this is done by means of the Verlet algorithm [(Eq. 1)](#eq-ver).

3. *Equilibrate the system*, i.e. allow it to reach thermal equilibrium. Altough it is really difficult to let the system staibilize at a *chosen target temperature*; this has to be with the fact that Newton's equation preserves the total energy of the system and thus the simulation is run in what is called the *microcanonical ensemble*. Temperature is an output of the system, not an input. In order to thermalize the system at a target temperature we need to move to the *canonical ensemble*. This is achieved by applying the Nosé–Hoover thermostat[$^{[2]}$](#bib-hoo), so that the system equilibrate at a target temperature. Care must be taken, since the thermostat changes the equation of motion; thus, in order to get meaningful results from the simulation, after the equilibration one must switch off the thermostat and go back to classical equation of motion. Hopefully, the temperature will then remain very close to the target one. 

4. *Compute the physical quantities of interest*. In MD the values of the physical quantities are extracted by performing averages over time and over the ensemble of particles. 

## *II.II* - COMPUTING THE FORCE FIELD. CAR-PARRINELLO MOLECULAR DYNAMICS
We recall from [(Eq. 1)](#eq-ver) that in order to compute the trajectory we need to evaluate at each timestep the forces acting on every particles. They are computed as the derivative of the total energy with respect to the coordinates,

<a id='eq-for'></a>
$$F_I = -\frac{\partial E(\{R\})}{\partial R_I} \tag{2}$$

In classical MD, the energy is computed as a sum of empirical potentials, that usually comprise for example a two-body pair potential (could be the Lennard-Jones potential) with a 3-body correction term and a mean-field term proportional to the density. The more the terms included, the more accurate the model is. 

**Ab initio Molecular Dynamics** consists of techniques that compute the force field *ab initio* starting from the electronic configurations around the nuclei. At every step, *Schroedinger equation* must be solved in order to find the electronic wavefunction and the energy of the system. Then, by means of the *Hellmann-Feynman theorem* we can compute its derivative in an efficient way to evaluate the forces [(Eq. 2)](#eq-for), so that we can evolve the ions coordinates. Nuclei are still considered as classical particles and obey Newton's equation, but in order to compute the force we need to solve quantum mechanical problem. *Born-Oppenheimer Molecular Dynamics* (BOMD) assumes that electrons stay in their ground state and that the electronic configuration only depends on the current atomic configurations (*adiabaticity assumption*), that is, we neglect every contributions to the dynamics that comes from "the history" of the electronic wavefunction.

Solving at each step the Schroedinger equation and then evolve the atomic positions is a huge and expensive computational task and hence ab-initio MD can simulate smaller systems for a reduced amount of time with respect to classical MD. However, since at each timestep the displacement of the nuclei is very small we can expect that the electronic wavefunction does not change that much every timestep and indeed we can take advantage of this fact to evolve the wavefunction. It is not necessary to solve the quantum mechanical probelm from scrtach every time. This is the fundamental assumption of *Car-Parrinello Molecular Dynamics* (CPMD)[$^{[3]}$](#bib-cpmd). We can calculate ”forces” over each point in the space of the wavefunctions and treat them as a classical fluid that is running after a potential that changes at each step, and propagate everything with the Verlet algorithm. It is sort of dynamical simulated annealing, as explained in the original paper. The "classical" Lagrangian used for "evolve" the wavefunction is

<a id='eq-cpla'></a>
$$\mathcal{L} = \sum_\nu \frac{\mu}{2}\int_\Omega d^3r \ |\dot\psi_\nu|^2 + \sum_I \frac{m_I}{2}\dot R_I^2 - E[\{\psi\}, \{R\}] + \sum_{ij}\Lambda_{ij}\left(\int_\Omega d^3r \ \psi_i^*\psi_j-\delta_{ij}\right) \tag{3}$$

where $\mu$ is the ”fake” mass of the electrons, $E$ is the energy functional and the last term represents the orthonormality constraints that keeps the electronic orbitals orthogonal to each other, $\Lambda_{ij}$ being the Lagrangian multipliers. This is in the end a classical Lagrangian, from which we can derive classical equation of motion using Euler-Lagrange equation

<a id='eq-eul'></a>
$$\frac{\partial \mathcal{L}}{\partial x_i} = \frac{d}{dt}\frac{\partial \mathcal{L}}{\partial \dot x_i} \tag{4}$$

Note here that the coordinates that we are using to derive the dynamics are the electron wavefuntions coordinates (one coordinate for the value and one for the time derivative for each point in the discretized space) and the atomic coordinates (usual velocity and positions), as well as the Lagrangian multipliers. The great improvement introduced by CPMD is now clear: the electronic wavefunctions and the atomic coordinates are evolved **simultaneously**, obeying the equations of motion

<a id='eq-cpmd'></a>
$$
\begin{align}
\mu \ddot\psi_\nu(r,t) &= -\frac{\delta E}{\delta \psi^*_\nu} + \sum_{j}\Lambda_{\nu j}\psi_j \tag{5.a}\\
M_I \ddot {\vec R_I} &= -\frac{\partial E}{\partial R_I} \tag{5.b}
\end{align}
$$

Both will be integrated using Verlet scheme [(Eq. 1)](#eq-ver). The first equation of motion [(Eq. 5.a)](#eq-cpmd) tells us that the electronic wavefunction oscillates around the ground state, following "adiabatically" the true minimum of the energy, so that BOMD and CPMD represents "on average" the same dynamics.

# *III* - CODE

In [3]:
work_dir = os.getcwd() + '/'

pack_dir = work_dir + 'packmol/' # packmol input directory
input_dir = work_dir + 'Input/' # input directory
out_dir = work_dir + 'Output/' # output directory
for wk_dir in [pack_dir, input_dir, out_dir]:
    try:
        os.mkdir(wk_dir)
    except:
        _ = None
        
pseudo_dir = work_dir + 'pseudo/' # pseudo directory
        
### SYSTEM ###
m_H = 1.0079 # u
m_O = 16 # u

N_H2O = 32 # number of H2O molecules
rho = 0.96 # density in g/cm^3

In [68]:
m_H2O = N_H2O*(2*m_H+m_O)*m_u # total mass of the system in g
L = (m_H2O/rho)**(1/3)/1e-8   # length of the side of the cube in Å

xyz_mol = 'water.xyz'     # input file with the coordinates of a single water molecule in Å
xyz_box = 'water_box.xyz' # output file with the coordinates of all water molecules in Å
pk_inp = 'water_box.inp'  # packmol input file
pk_tol = 2.               # tolerance

def gen_xyz_mol(filename): # generate the `xyz_mol` file
    # SPC water model
    theta = 104.5 #°
    d = 1 # Å

    ## Coordinates of the water molecule.
    r_O = np.zeros(3) + [1,1,1]
    r_H1 = r_O + [d*np.cos(theta/2*np.pi/180), d*np.sin(theta/2*np.pi/180), 0]
    r_H2 = r_O + [d*np.cos(theta/2*np.pi/180), -d*np.sin(theta/2*np.pi/180), 0]
    
    with open(filename, 'w') as f:
        print('3\n', file=f)
        print(f'O    {r_O[0]:.4f}    {r_O[1]:.4f}    {r_O[2]:.4f}', file=f)
        print(f'H    {r_H1[0]:.4f}    {r_H1[1]:.4f}    {r_H1[2]:.4f}', file=f)
        print(f'H    {r_H2[0]:.4f}    {r_H2[1]:.4f}    {r_H2[2]:.4f}', file=f)

def gen_pk_inp(filename):
    with open(pack_dir+pk_inp, 'w') as f:
        print(f'tolerance {pk_tol:.2f}', file=f)
        print('filetype xyz', file=f)
        print(f'output {xyz_box}', file=f)
        print('', file=f)
        print(f'structure {xyz_mol}', file=f)
        print(f'  number {N_H2O}', file=f)
        print(f'inside cube 0. 0. 0. {L-pk_tol:.2f}', file=f)
        print('end structure', file=f)
  
def gen_xyz_box():
    gen_xyz_mol(pack_dir+xyz_mol)
    gen_pk_inp(pack_dir+pk_inp)  
    
    pack_cmd = '$HOME/Programs/packmol-20.14.0/bin/packmol'
    os.system(f'cd {pack_dir} && {pack_cmd} < {pk_inp} > packmol.out')
    
def read_init_pos():
    with open(pack_dir+xyz_box, 'r') as f:
        init_pos = f.readline()
        f.readline()
        init_pos += f'Lattice="{L:.2f} 0.0 0.0 0.0 {L:.2f} 0.0 0.0 0.0 {L:.2f}"\n'
        init_pos += f.read()
    return init_pos

comments = {'CONTROL' : 
                {'title' : 'title of the run',
                 'calculation' : 'type of calculation to be performed',
                 'isave' : 'frequency for writing the restart file',
                 'restart_mode' : 'how to start the simulation',
                 'nstep' : 'number of Car-Parrinello steps performed in this run',
                 'iprint' : 'frequency for writing the relevant quantities',
                 'tprnfor' : 'print forces',
                 'dt' : 'timestep in atomic units',
                 'outdir' : 'output directory',
                 'prefix' : 'prepend the prefix to input/output filenames and restart folders',
                 'ndr' : 'number directory read',
                 'ndw' : 'number directory write',
                 'etot_conv_thr' : 'convergence threshold on total energy for ionic minimization',
                 'forc_conv_thr' : 'convergence threshold on forces for ionic minimization',
                 'ekin_conv_thr' : 'convergence criterion for electron minimization',
                 'pseudo_dir' : 'pseudopotential directory'},
           
           'SYSTEM' : 
                {'ibrav' : 'Bravais lattice',
                 'celldm(1)' : "'a' crystallographic constant in a.u.",
                 'nat' : 'number of atoms',
                 'ntyp' : 'type of atoms',
                 'ecutwfc' : 'kinetic energy cutoff (Ry) for wavefunctions'},
           
            'ELECTRONS' : 
                {'electron_dynamics' : 'set how electrons are moving'},
           
            'IONS' : 
                {'ion_dynamics' : 'set how ions are moving',
                 'ion_velocities' : 'ionic velocities',
                 'tempw' : 'ionic temperature in K'}}

def make_input_file(filename, com_space=40, **kwargs):
    inp_file = ""
    for nml in kwargs:
        inp_file += f"&{nml}\n"
        for key,val in kwargs[nml].items():
            inp_str = f"  {key} = {val},"
            inp_file += inp_str + ' '*(com_space-len(inp_str)) + f" ! {comments[nml][key]}\n"
        inp_file += '/ \n\n'
    
    inp_file += "ATOMIC_SPECIES\n" +\
               f"  O  {m_O:.3f}d0  O_ONCV_PBE-1.2.upf\n" +\
               f"  H  {m_H:.4f}d0  H_ONCV_PBE-1.2.upf\n\n"
    
    init_pos = read_init_pos()
    inp_file += "ATOMIC_POSITIONS (angstrom)\n"
    inp_file += init_pos[init_pos.find('"\n')+2:]
    
    with open(input_dir+filename, 'w') as f:
        f.write(inp_file)
        
    return inp_file

In order to run a simulation we will usage the `cp.x` package from Quantum Espresso[$^{[4,5]}$](#bib-qe1). We have to specify the parameters of the computation in an input file, which is organized into several namelists, followed by other fields (“cards”) introduced by keywords. The namelists are:
- `&CONTROL`: general variables controlling the run, for example the title, the type of calculation, the path to the needed directories, as well as the number of steps and the timestep of the run.
- `&SYSTEM`: structural information on the system under investigation. The system comprises $32$ water molecules placed in a cubic box of length around $10$ Å so that to match the density $0.96$ g/cm$^3$.
- `&ELECTRONS`: set the electronic variables and the electron dynamics.
- `&IONS`: set ionic variables and dynamics.

After the namelists, you have several fields (“cards”) introduced by keywords with self-explanatory names:
- `ATOMIC_SPECIES`: it is specified the pesudopotentials used for each atom type"
- `ATOMIC_POSITIONS`: the coordinates of the atoms. The initial disposition of the water molecules inside the simulation box is generated by the package *packmol*[$^{[6]}$](#bib-pack)."

In [69]:
gen_xyz_box()

## III.I REACHING THE ELECTRONIC GROUND STATE - BOMD

It is not a good idea to start immediately with a CPMD simulation because if you start assigning random velocities to the atoms, you can have some sharp changes in the atomic velocities because they could be not consistent with the current atomic positions, or because two atoms could be too near to each other; this will reflect in little kinks to the nuclei and by consequence little kinks to the electronic degrees of freedom, that will cause a little more kinetic energy to the electronic system (that we want as cold as possible) and a little rise in the electronic temperature. Thus the first run, when starting from scratch, is always an electronic minimization, with fixed ions and cell, to bring the electronic system on the ground state relative to the starting atomic configuration, and we are going to do it by means of the *conjugated gradient* method.

In [70]:
## Produce the BOMD input file
bomd_inp = 'cp.water.bomd.in' # name of the file
print_input_file = True # wheter to print the produced file at the end of the subroutine

inp_key = {"&CONTROL" :
               {"title" : f"'Water {N_H2O} molecules'",
                "calculation" : "'cp'",
                "restart_mode" : "'from_scratch'",
                "ndw" : 50,
                "nstep" : 100,
                "iprint" : 10,
                "isave" : 20,
                "tprnfor" : ".TRUE.",
                "dt" : "5.00d0",
                "etot_conv_thr" : "1.0d-5",
                "ekin_conv_thr" : "1.0d-5",
                "forc_conv_thr" : "1.0d-3",
                "prefix" : 'H2O',
                "pseudo_dir" : f"'{pseudo_dir}'",
                "outdir" : f"'{out_dir}'"},
          
           "&SYSTEM" :
               {"ibrav" : "1",
                "celldm(1)" : f"{L/a0:.2f}",
                "nat" : N_H2O*3,
                "ntyp" : 2,
                "ecutwfc" : "60.00"},
          
           "&ELECTRONS" :
               {"electron_dynamics" : "'cg'"},
          
           "&IONS" : 
               {"ion_dynamics" : "'verlet'",
                "ion_velocities" : "'random'",
                "tempw" : "400.00d0"}}

inp_file = make_input_file(bomd_inp, CONTROL=inp_key["&CONTROL"],  SYSTEM=inp_key["&SYSTEM"],  
                                     ELECTRONS=inp_key["&ELECTRONS"], IONS=inp_key["&IONS"])
    
## Print the input file
if print_input_file==True: print(inp_file, end='')

&CONTROL
  title = 'Water 32 molecules',          ! title of the run
  calculation = 'cp',                    ! type of calculation to be performed
  restart_mode = 'from_scratch',         ! how to start the simulation
  ndw = 50,                              ! number directory write
  nstep = 100,                           ! number of Car-Parrinello steps performed in this run
  iprint = 10,                           ! frequency for writing the relevant quantities
  isave = 20,                            ! frequency for writing the restart file
  tprnfor = .TRUE.,                      ! print forces
  dt = 5.00d0,                           ! timestep in atomic units
  etot_conv_thr = 1.0d-5,                ! convergence threshold on total energy for ionic minimization
  ekin_conv_thr = 1.0d-5,                ! convergence criterion for electron minimization
  forc_conv_thr = 1.0d-3,                ! convergence threshold on forces for ionic minimization
  prefix = H2O,                

In [ ]:
## BOMD RUN
n_mpi = 2
n_omp = 1
cmd_mpirun = f'mpirun -n {n_mpi} -genv OMP_NUM_THREADS={n_omp} -genv I_MPI_PIN_DOMAIN=omp '
#os.system(cmd_mpirun + f'cp.x -in {input_dir+bomd_inp}')

### III.I.I READING OUTPUT FILES

In [15]:
def get_evp():
    return pd.read_csv(out_dir+'H2O.evp',  names=['', 'time (ps)', 'ekinc', 'T_ion (K)', 'etot', 'econs', 'econt'], 
                       index_col=0,  usecols=[0,1,2,4,5,7,8],  delim_whitespace=True,  comment='#')

def get_nstep():
    evp = get_evp()
    return evp.shape[0]+1

def get_time(units='ps'):
    units_dict = {'fs':1e3, 'ps':1, 'ns':1e-3, 'μs':1e-6, 'ms':1e-9}
    nstep = get_nstep()
    
    time = np.zeros(nstep)
    time[1:] = evp['time (ps)'].values * units_dict[units]
    return time

def read_positions():
    nstep = get_nstep()
    pos = np.zeros((nstep, N_H2O*3, 3))
    
    init_pos = read_init_pos()
    for j,poss in enumerate(init_pos.split('\n')[2:-1]):
        pos_tmp = poss.split()
        pos[0,j,:] = float(pos_tmp[1]), float(pos_tmp[2]), float(pos_tmp[3])
        
    with open(out_dir+'H2O.pos', 'r') as f:
        for i in range(1,nstep):
            f.readline() # discard the first line
            for j in range(3*N_H2O):
                pos_tmp = f.readline().split()
                pos[i,j,:] = float(pos_tmp[0])*a0, float(pos_tmp[1])*a0, float(pos_tmp[2])*a0
    return pos

def read_velocities():
    nstep = get_nstep()
    vel = np.zeros((nstep, N_H2O*3))

    with open(out_dir+'H2O.vel', 'r') as f:
        for i in range(1,nstep):
            f.readline() # discard the first line
            for j in range(N_H2O*3):
                vel_tmp = f.readline().split()
                vel[i,j] = abs(np.mean([float(vel_tmp[k]) for k in range(3)]))
    return vel
                
def read_forces():
    nstep = get_nstep()
    forces = np.zeros((nstep, N_H2O*3))

    with open(out_dir+'H2O.for', 'r') as f:
        for i in range(1,nstep):
            f.readline() # discard the first line
            for j in range(N_H2O*3):
                forces_tmp = f.readline().split()
                forces[i,j] = abs(np.mean([float(forces_tmp[k]) for k in range(3)]))
    return forces

def write_trajectory():
    init_pos = read_init_pos()
    pos = read_positions()
    
    with open(out_dir+'H2O.traj.xyz', 'w') as f:
        for i in range(len(pos)):
            f.write(init_pos.split('\n')[0] + '\n' + init_pos.split('\n')[1] + '\n') # xyz header line
            for j,(x,y,z) in enumerate(pos[i]):
                if j%3==0: 
                    f.write(f'  O           {x:9.6f}       {y:9.6f}       {z:9.6f}\n')
                else:
                    f.write(f'  H           {x:9.6f}       {y:9.6f}       {z:9.6f}\n')

The file `H2O.evp` contains some thermodynamics data:
- `ekinc` is the electrons fictitious kinetic energy. In this case it is zero because we are not running yet the Verlet algorithm, and there is no lagrangian yet that can be used to define an electronic classical (fake) kinetic energy.
- `T_ion` is the temperature of the ion, computed as $T = \frac{2}{3Nk_B}\sum_I\frac{m_Iv_I^2}{2}$.
- `etot` is the DFT (potential) energy of the system.
- `econs` is the potential energy plus the kinetic energy of the nuclei and it is a physically meaningful constant of motion in the limit of zero electronic fictitious mass.
- `econt` is the constant of motion of the CP Lagrangian [(Eq. 3)](#eq-cpla). If the time step $dt$ is small enough this will be up to a very good precision a constant, although it is not a physical quantity, since the "fake" kinetic energy defined in $\mathcal{L}$ has nothing to do with the quantum kinetic energy of the electrons.

In [19]:
## THERMODYNAMICS
evp = get_evp()

time_units = 'fs'
time = get_time(time_units)

### PLOT
fig, ax = plt.subplots(1,2, figsize=(15,6), constrained_layout=True)

# temperature
ax[0].plot(time[1:], evp['T_ion (K)'], '.-')
ax[0].set(xlim=(0,time[1]+time[-1]), ylim=(0,None), xlabel=r'$t\ ({})$'.format(time_units), ylabel=r'$T\ (K)$')

# energy
ax[1].plot(time[1:], evp['etot'], '.-', label='etot')
ax[1].plot(time[1:], evp['econs'], '.-', label='econs')
ax[1].plot(time[1:], evp['econt'], '.-', label='econt')
ax[1].set(xlim=(0,time[1]+time[-1]), xlabel=r'$t\ ({})$'.format(time_units), ylabel=r'$E\ (a.u.)$')
ax[1].legend(fontsize=14, fancybox=True, shadow=True)

plt.show()

The files `H2O.pos`, `H2O.vel` and `H2O.for` contain, respectively, the positions, the velocities and the forces of every atom in the simulation box. The following code also produces a trajectory file (in "extended xyz" format [$^{[7]}$](#bib-xyz)) named `H2O.traj.xyz`.

In [31]:
## Write the trajectory file in an "extended xyz" format.
write_trajectory() 

# Then, it can be visualized with some molecule viewers software as`ovito`, 'VMD' or 'Jmol'.
view_cmd = '$HOME/Programs/ovito-3.8.3/bin/ovito'
os.system(view_cmd+' Output/H2O.traj.xyz')

qt.qpa.plugin: Could not load the Qt platform plugin "wayland" in "" even though it was found.


0

In [25]:
## VELOCITIES AND FORCES
velocities = read_velocities()
forces = read_forces()

time_units = 'fs'
time = get_time(time_units)

## PLOT
fig, ax = plt.subplots(1, 2, figsize=(15,6), constrained_layout=True)

for i in range(N_H2O*3): ax[0].plot(time[1:], velocities[1:,i], 'D', ms=4., alpha=0.1, c='blue')
ax[0].set(xlim=(0,time[1]+time[-1]), ylim=(0,None), 
          xlabel=r'$t\ ({})$'.format(time_units), ylabel=r'$v\ (a.u.)$')
    
for i in range(N_H2O*3): ax[1].plot(time[1:], forces[1:,i], 'D', ms=4., alpha=0.1, c='red')
ax[1].plot(time[1:], np.mean(forces[1:,:], axis=1), '--', lw=1., c='red') # mean value of the forces>
ax[1].set(xlim=(0,time[1]+time[-1]), ylim=(2e-6,1), 
          xlabel=r'$t\ ({})$'.format(time_units), ylabel=r'$F\ (a.u.)$', yscale='log')

plt.show()

## III.II REACHING A GOAL TEMPERATURE - SIMULATING THE CANONICAL ENSEMBLE USING NOSÈ-HOOVER

...

# *IV* - RESULTS

### MEAN-SQUARED DISPLACEMENT

In [ ]:
msd = np.zeros((nstep, 2))
for k in range(nstep):
    msd_O, msd_H = 0, 0
    for i in range(N_H2O):
        msd[k,0] += np.linalg.norm(poss[k,3*i,:]-poss[0,3*i,:])**2/N_H2O
        
        msd[k,1] += np.linalg.norm(poss[k,3*i+1,:]-poss[0,3*i+1,:])**2/N_H2O/2
        msd[k,1] += np.linalg.norm(poss[k,3*i+2,:]-poss[0,3*i+2,:])**2/N_H2O/2

# *V* - CONCLUSIONS

---
# APPENDIX

# BIBLIOGRAPHY
1. *L. Verlet*, [Phys. Rev. **159**, 98 (1967)](https://doi.org/10.1103/PhysRev.159.98).<a id='bib-ver'></a>
2. *W. G. Hoover*, [Phys. Rev. A **31**, 1695 (1985)](https://doi.org/10.1103/PhysRevA.31.1695).<a id='bib-hoo'></a>
3. *R. Car and M. Parrinello*, [Phys. Rev. Lett. **55**, 2471 (1985)](https://doi.org/10.1103/PhysRevLett.55.2471).<a id='bib-cpmd'></a>
4. *P. Giannozzi et al.*, [J. Phys.: Condens. Matter 21, 395502 (2009)](https://doi.org/10.1088/0953-8984/21/39/395502)<a id='bib-qe1'></a>
5. *P. Giannozzi et al.*, [J. Phys.: Condens. Matter 29, 465901 (2017)](https://doi.org/10.1088/1361-648X/aa8f79)<a id='bib-qe2'></a>
6. *L. Martínez, R. Andrade, E. G. Birgin and J. M. Martínez*, [J. Comput. Chem. 30(13):2157-2164 (2009)](https://doi.org/10.1002/jcc.21224).<a id='bib-pack'></a>
7. XYZ file format, [Wikipedia page](https://en.wikipedia.org/wiki/XYZ_file_format).<a id='bib-xyz'></a>